`Author: Victor Radermecker | Date: 11/10/2021 | Project: Master Thesis`

## Imports

In [14]:
import branca.colormap as cm
import folium
import geopandas as gpd
import pandas as pd
import pyproj
import shapely.ops as sp_ops
from pyproj import Transformer
from tqdm.auto import tqdm

tqdm.pandas()
pd.set_option("display.max_columns", 100)

## Global Functions and variables

In [15]:
belgian_coords = 31370
gps_coords = 4326

In [6]:
def transform_geom_to_srid(geom, scrs, tcrs):
    """ Transform a geometry to a new target CRS.
        Works with pyproj version >= 2.x.x
        - geom is a Shapely geometry instance
        - scrs is your input CRS EPSG integer (the one of your original data)
        - tcrs is your target CRS EPSG integer (the one you want to reproject
                  your data in, probably 4326 in your case)
    """
    project = Transformer.from_crs(
        "EPSG:" + str(scrs), "EPSG:" + str(tcrs), always_xy=True
    )
    return sp_ops.transform(project.transform, geom)

## Statistical sectors - Preprocessing

In [9]:
# Statistical Sectors
fname = (
    "../data/maps/stat_sectors/shapefile/sh_statbel_statistical_sectors_20210101.shp"
)

# Read and display
ss = gpd.read_file(fname)

# Keep only statistical sectors from RBC and rename 'CS01012021' column to 'Sector'
ss = ss[ss["T_REGIO_FR"] == "Région de Bruxelles-Capitale"]
ss = ss[["CS01012021", "T_REGIO_FR", "T_SEC_FR", "geometry"]]
ss.rename(columns={"CS01012021": "Sector"}, inplace=True)
print("Number of statistical sectors for RBC: ", len(ss))

ss.head()

Number of statistical sectors for RBC:  724


,Sector,T_REGIO_FR,T_SEC_FR,geometry
1997,21001A00-,Région de Bruxelles-Capitale,RESISTANCE,"POLYGON Z ((146125.190 169525.098 0.000, 14612..."
1998,21001A011,Région de Bruxelles-Capitale,KLEINMOLEN,"POLYGON Z ((146420.844 169763.840 0.000, 14647..."
1999,21001A02-,Région de Bruxelles-Capitale,WAYEZ,"POLYGON Z ((146389.484 169587.606 0.000, 14638..."
2000,21001A031,Région de Bruxelles-Capitale,RAUTER-SUD,"POLYGON Z ((145888.469 169013.527 0.000, 14584..."
2001,21001A041,Région de Bruxelles-Capitale,VEEWEYDE-SUD,"POLYGON Z ((145327.859 169178.606 0.000, 14533..."


#### Start conversion 

This cell starts the conversion from the Belgian coordinate system to the universal GPS system.

<div class="alert alert-block alert-warning">
Warning: Takes about an hour (for whole Belgium). RBC is faster.
</div>

In [ ]:
ss["geometry"] = ss["geometry"].progress_apply(
    transform_geom_to_srid, args=(belgian_coords, gps_coords,)
)

In [ ]:
save_json = True
save_shp = False

# Save to GeoJson file for choropleth maps
if save_json:
    ss.to_file(
        "../data/maps/stat_sectors/json/RBC_stat_sectors_2021.json", driver="GeoJSON"
    )

if save_shp:
    ss.to_file("../data/maps/stat_sectors/shapefile/RBC_stats_sectors_2021_gps.shp")

## Proximus Neighborhoods - Preprocessing

In [16]:
# Statistical Sectors
fname = "../data/maps/prox_neighbor/shapefile/UrbAdm_MONITORING_DISTRICT.shp"

# Read and display
pn = gpd.read_file(fname)

pn.head()

,ID,VERSIONID,MDRC,NAME_FRE,NAME_DUT,NAME_BIL,AREA,INSPIRE_ID,BEGIN_LIFE,END_LIFE,geometry
0,83.0,1.0,83,CONSCIENCE,CONSCIENCE,CONSCIENCE,469489.90,BE.BRUSSELS.BRIC.ADM.MD.53,2015-10-05,None,"POLYGON Z ((151836.566 173466.414 0.000, 15186..."
1,80.0,1.0,80,HELMET,HELMET,HELMET,718634.24,BE.BRUSSELS.BRIC.ADM.MD.54,2015-10-05,None,"POLYGON Z ((150764.790 173563.033 0.000, 15080..."
2,19.0,1.0,19,VIEUX LAEKEN OUEST,OUD LAKEN WEST,OUD LAKEN WEST / VIEUX LAEKEN OUEST,499645.52,BE.BRUSSELS.BRIC.ADM.MD.123,2015-10-05,None,"POLYGON Z ((147858.070 174004.707 0.000, 14786..."
3,20.0,1.0,20,VIEUX LAEKEN EST,OUD LAKEN OOST,OUD LAKEN OOST / VIEUX LAEKEN EST,1040449.92,BE.BRUSSELS.BRIC.ADM.MD.66,2015-10-05,None,"POLYGON Z ((148132.120 173445.561 0.000, 14813..."
4,800.0,1.0,800,INDUSTRIE NORD,INDUSTRIE NOORD,INDUSTRIE NOORD / NORD,6733601.22,BE.BRUSSELS.BRIC.ADM.MD.70,2018-01-02,None,"POLYGON Z ((151113.691 177049.939 0.000, 15111..."


#### Start conversion 

This cell starts the conversion from the Belgian coordinate system to the universal GPS system.

In [17]:
pn["geometry"] = pn["geometry"].progress_apply(
    transform_geom_to_srid, args=(belgian_coords, gps_coords,)
)

  0%|          | 0/145 [00:00<?, ?it/s]

In [18]:
save_json = True
save_shp = False

# Save to GeoJson file for choropleth maps
if save_json:
    pn.to_file(
        "../data/maps/prox_neighbor/json/RBC_Neighborhoods_gps.json", driver="GeoJSON"
    )

if save_shp:
    pn.to_file("../data/maps/prox_neighbor/shapefile/RBC_Neighborhoods_gps.json")